<a href="https://colab.research.google.com/github/anilaksu/Algorithmic-Trading-Codes/blob/main/Udemy_Triangular_Arbitrage_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Triangular Arbitrage for Crypto with Python Udemy Training**


Anil Aksu

Personal e-mail: aaa293@cornell.edu



**Outline:**


1.  CEFI Triangular Arbitrage
  * Synchronous vs Asynchronous
  * Blocking & Timeouts
  * Scraping with Selenium
  * Asynchronous Functions
  * Asynchronous Iterators
2. DEFI Triangular Arbitrage
  * Coroutine Objects and Async Functions
  * Ways of running coroutines
  * Cancelling coroutines
  * Awaitable Objects



Clone GitHub Repo:

git clone https://github.com/CryptoWizardsNet/poloniex-tri-arb



If you have never used GitHub before or do not have GitHub working on your command line:

https://www.youtube.com/watch?v=CKcqniGu3tA

https://git-scm.com/book/en/v2/Getting-Started-Installing-Git



Google Colab Notebook:

https://colab.research.google.com/drive/1gvX613_dwF5p6Hea9xMYeC-KokE9e9C_?usp=sharing



Poloniex latest docs:

https://docs.poloniex.com/#introduction



Binance latest docs:

https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/ColabNotebooks/FinanceAlgorithms
!ls # special shell command to view the files in the home directory of the notebook environment

Mounted at /content/gdrive
/content/gdrive/MyDrive/ColabNotebooks/FinanceAlgorithms
 2013-03-08options.csv	      EURUSD_Options_Data.csv	   OptionsTrading.ipynb
 2013-03-08stocks.csv	      EURUSD_Options_Data.gsheet   PriceJump.gdraw
'Asynchronous Python.ipynb'  'ForEx&IndexData.xls'	  'Stock Markets Codes.ipynb'
 async_scrape.py	      local.csv


#**1. How to pull any API Data**

In [ ]:
# Here we install required libraries for API data manipulation
!python3 -V
!which pip3
!pip3 install requests --upgrade --no-cache-dir

Python 3.10.12
/usr/local/bin/pip3


In [2]:
# Here we import required libraries
import requests
import json

In [ ]:
# Poloniex prices
prices = requests.get("https://api.poloniex.com/markets/price")
prices_list = []

if prices.status_code == 200:
  prices_json = json.loads(prices.text)
  for p in prices_json:
    prices_list.append({"ticker": p["symbol"], "price" : p["price"]})

In [ ]:
print(prices_list)

[{'ticker': 'BTS_BTC', 'price': '0.0000000625'}, {'ticker': 'DASH_BTC', 'price': '0.000624'}, {'ticker': 'DOGE_BTC', 'price': '0.0000025'}, {'ticker': 'LTC_BTC', 'price': '0.001401'}, {'ticker': 'XLM_BTC', 'price': '0.00000224'}, {'ticker': 'XEM_BTC', 'price': '0.000000515'}, {'ticker': 'XMR_BTC', 'price': '0.002226'}, {'ticker': 'XRP_BTC', 'price': '0.00000983'}, {'ticker': 'BTC_USDT', 'price': '64959.13'}, {'ticker': 'DASH_USDT', 'price': '39.5'}, {'ticker': 'LTC_USDT', 'price': '91.135'}, {'ticker': 'XLM_USDT', 'price': '0.1454'}, {'ticker': 'XMR_USDT', 'price': '144.98'}, {'ticker': 'XRP_USDT', 'price': '0.6387'}, {'ticker': 'ETH_BTC', 'price': '0.05428'}, {'ticker': 'ETH_USDT', 'price': '3524.25'}, {'ticker': 'SC_BTC', 'price': '0.00000014'}, {'ticker': 'DCR_BTC', 'price': '0.000402'}, {'ticker': 'LSK_BTC', 'price': '0.0000185'}, {'ticker': 'STEEM_BTC', 'price': '0.00000459'}, {'ticker': 'ETC_BTC', 'price': '0.000528'}, {'ticker': 'ETC_ETH', 'price': '0.00965'}, {'ticker': 'ETC_US

In [ ]:
# Poloniex Orderbook
ticker = "DASH_BTC"
limit = 20
# Structure Orderbook
ask_prices = []
ask_sizes = []
counts = 0

ob = requests.get(f"https://api.poloniex.com/markets/{ticker}/orderBook?limit={limit}")

if ob.status_code == 200:
  ob_json = json.loads(ob.text)
  for a in ob_json["asks"]:
    if counts% 2 == 0:
      ask_prices.append(a)
    else:
      ask_sizes.append(a)
    counts += 1
print(ask_prices)
print(ask_sizes)

['0.000625', '0.000627', '0.000629', '0.000649', '0.00065', '0.000658', '0.000667', '0.000668', '0.000679', '0.00068', '0.000684', '0.000687', '0.000689', '0.000694', '0.000698', '0.0007', '0.000705', '0.00072', '0.000728', '0.000731']
['3.19', '25.87', '1.84', '0.05', '40', '0.74', '0.08', '0.99', '0.05', '0.07', '0.10', '0.02', '0.10', '0.10', '0.72', '0.02', '0.02', '2.27', '0.06', '0.04']


In [ ]:
# Binance Historical Data
symbol = "BTCUSDT"
interval = "1d"
candles = requests.get(f"https://data.binance.us/api/v3/klines?symbol={symbol}&interval={interval}")
candles_json = []

if candles.status_code == 200:
  candles_json = json.loads(candles.text)
  print(candles_json)

#2. Structuring Pairs



*   Step 0: Gather Correct Coins
*   Step 1:

https://docs.poloniex.com#introduction



In [3]:
import requests
import json

In [12]:
def get_coin_tickers(url):
  '''
      This function returns the trade activities at the given url
  '''
  req = requests.get(url)
  coin_json = json.loads(req.text)
  return coin_json

def collect_tradeables(coin_json):
  '''
      This function returns the tradeable pairs at the given json
  '''
  coin_list = set()
  # List of all tradeable coins
  for coin in coin_json:
    #print(coin)
    coin_list.add(coin['symbol'])

  coin_list = list(coin_list) # Here we convert to list
  return coin_list


In [19]:
def structure_triangular_pairs(coin_list):
  '''
      This function returns the list of pairs that form triangule
  '''
  # Declare Variables
  triangular_pairs_list = []
  remove_duplicates_list = []
  pairs_list = []

  # Get Pair A
  for pair_a in coin_list:
    pair_a_split = pair_a.split("_")
    a_base = pair_a_split[0]
    a_quote = pair_a_split[1]
    print(a_base, a_quote)
  #return structured_pairs

In [20]:
# Extract list of coins and prices from Exchange
url = "https://api.poloniex.com/markets/price"

coin_json = get_coin_tickers(url)          # Here we get the json format of trade activities at the given url
coin_list = collect_tradeables(coin_json)  # Here we get the list of coins
structure_triangular_pairs(coin_list)
#print(coin_list)

CSAS USDT
DHT USDT
KLV TRX
BNX USDT
CITY USDD
ZBC USDT
TRX USDD
KAS USDT
GRT USDT
SAFEREUM USDT
KP3R USDT
CATS USDT
BFIC USDT
BSWAP USDT
PERP USDT
GLM USDT
FRONT USDT
SWFTC USDT
UNI USDT
ETHW USDD
AGI USDT
RATS USDT
DORKL USDT
BLY USDT
ELON USDD
GDX USDT
WPEPE USDT
QTUM BTC
SORA USDT
SAITAMA USDT
XRP TRX
JUP USDT
POR USDD
POLOTEST2 POLOTEST1
SSV USDT
DCR USDT
JUV USDT
NEVER USDT
DC USDT
QTUM USDT
PONK USDT
BOO USDT
ETHF USDT
BNB USDC
FTM BTC
MUSKX USDT
APX1 USDT
KON USDT
DOGE BTC
INJ USDT
ETC USDT
BTC TUSD
TORN USDT
BITCI USDT
LOOT USDT
INS USDT
FLOKI USDT
NEO USDT
WLKN USDT
ADA BTC
AFC USDT
PORTO USDD
SFARM USDT
ROUP USDT
GMEE USDT
LZM USDT
POP USDT
ID USDT
WFAI USDT
PNDC USDT
IMX USDT
OX USDT
SHEPE USDT
AC USDT
MASK USDD
SNX TRX
COMP ETH
KCS USDT
MATTER USDT
XMR USDT
BICO USDT
POPCAT USDT
MNDE USDT
LPT USDT
CGPT USDT
CHOW USDT
NFT USDD
ANALOS USDT
AI1 USDT
RDNT USDT
WBTC USDT
COCO USDT
FOOM USDT
LRC BTC
CEL USDD
LTC BTC
YFII USDT
POLOTEST2 POLOTEST4
DSWP USDT
XEC USDT
XRP USDC
PIXEL 

##1.5 Asynchronous Iterators

In [ ]:
# The Asynchronous Generators
import asyncio

async def download(urls):
  for url in urls:
    await asyncio.sleep(1)
    response = {'status': 200, 'data': f'content of {url}'}
    if url == 'bing.com':
      response['status'] = 500
    yield response

async def main():
  urls = [
      'google.com',
      'bing.com',
      'duckduckgo.com'
  ]

  responses = [value async for value in download(urls) if value['status'] != 200]
  print(responses)

await main()

[{'status': 500, 'data': 'content of bing.com'}]


#**2. Coroutines & Awaitables**

##2.1 Coroutines Objects and Async Functions

In [ ]:
import asyncio
import inspect

async def main():
  pass

print(type(main))
print(inspect.iscoroutinefunction(main))
print(dir(main()))

<class 'function'>
True
['__await__', '__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'cr_await', 'cr_code', 'cr_frame', 'cr_origin', 'cr_running', 'send', 'throw']


<ipython-input-143-56483b0f906b>:9: RuntimeWarning: coroutine 'main' was never awaited
  print(dir(main()))


##2.2 Ways of Running Coroutines

In [ ]:
import asyncio
import inspect

async def main():
  print('The main function')

# 1. using await
await main()
# 2. creating a task
asyncio.create_task(main())

The main function


<Task pending name='Task-58' coro=<main() running at <ipython-input-145-bc733164109c>:4>>

The main function


##2.3 Cancelling Coroutines

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while True:
      await asyncio.sleep(1)
      count += 1
      print(count)

async def main():
  task = asyncio.create_task(stopwatch())
  await asyncio.sleep(3)
  task.cancel()   # Here we say "Cancel the task after 3rd second"

await main()

1
2


##2.4 Awaitable Objects

In [ ]:
import asyncio

class Stopwatch:
  def __await__(self):
    yield

async def main():
  await Stopwatch()

await main()

#**3. Tasks, Futures and the Event Loop**

##3.1 Using Task Objects

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  task = asyncio.create_task(stopwatch())
  task.set_name('My Task')
  task.add_done_callback(callb)
  print(task.get_name())
  print(task.get_coro())
  print(task.get_name())
  await task

await main()

My Task
<coroutine object stopwatch at 0x7ea2787a4660>
My Task
1
2
3
4
task is done <Task finished name='My Task' coro=<stopwatch() done, defined at <ipython-input-150-b87e2fe8d0a6>:3> result=None>


##3.2 Interacting with the Event Loop:

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  print(asyncio.get_running_loop())
  task = asyncio.create_task(stopwatch())
  task.add_done_callback(callb)

  await task

await main()

<_UnixSelectorEventLoop running=True closed=False debug=False>
1
2
3
4
task is done <Task finished name='Task-69' coro=<stopwatch() done, defined at <ipython-input-151-f16058d61056>:3> result=None>


##3.3 The concept of the future:

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  task = asyncio.create_task(stopwatch())
  task.add_done_callback(callb)
  print(asyncio.isfuture(task))

  await task

await main()

True
1
2
3
4
task is done <Task finished name='Task-71' coro=<stopwatch() done, defined at <ipython-input-152-fd9b0fb67711>:3> result=None>
